#### Importing needed libraries

In [24]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras as extras

#### Creating a connection to the default postgres database

Take note of the use of the try and except block. This is done so as to catch any errors that may occur during the execution of the code block.

In [3]:
try:
    conn = psycopg2.connect(host="localhost", database="postgres", user="postgres", password="Databishop")
except psycopg2.Error as e:
    print(e)

In [4]:
# Using the connection to get a cursor to be used for makign queries
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)

# Setting autocommit for queries to True    
conn.set_session(autocommit=True)

#### Creating the LEGO Database

In [5]:
# Creating Database with name legoDB
try:
    cur.execute("CREATE DATABASE legodb")
except psycopg2.Error as e:
    print(e)

database "legodb" already exists



#### Reconnecting to the LEGO Database

In [6]:
# Clsoing the initial connection to the default database
try:
    conn.close()
except psycopg2.Error as e:
    print(e)

# Connecting to the LEGO Database    
try:
    conn = psycopg2.connect(host="localhost", database="legodb", user="postgres", password="Databishop")
except psycopg2.Error as e:
    print(e)

# Getting the cursor    
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print(e)
    
# Setting autocommit to True
conn.set_session(autocommit=True)

#### Creating the inventories table according to the defined schema

In [8]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS inventories \
        (id INTEGER PRIMARY KEY,\
        version INTEGER,\
        set_number VARCHAR(50));")
except psycopg2.Error as e:
    print(e)

#### Creating the colours table according to the defined schema

In [9]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS colours \
        (id INTEGER PRIMARY KEY,\
        name VARCHAR(50),\
        rgb VARCHAR(6),\
        is_trans BOOLEAN );")
except psycopg2.Error as e:
    print(e)

#### Creating the parts table according to the defined schema

In [11]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS parts \
    (part_number VARCHAR(50) PRIMARY KEY,\
    name TEXT,\
    part_cat_id INTEGER);")
except psycopg2.Error as e:
    print(e)

#### Creating the sets table according to the defined schema

In [12]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS sets \
    (set_number VARCHAR(10) PRIMARY KEY,\
    name VARCHAR(50),\
    year INTEGER,\
    theme_id INTEGER,\
    num_parts INTEGER);")
except psycopg2.Error as e:
    print(e)

#### Creating the themes table according to the defined schema

In [13]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS themes \
        (id INTEGER PRIMARY KEY,\
        name VARCHAR(50),\
        parent_id INTEGER);")
except psycopg2.Error as e:
    print(e)

#### Creating the part categories table according to the defined schema

In [19]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS part_categories \
        (id INTEGER PRIMARY KEY,\
        name VARCHAR(50));")
except psycopg2.Error as e:
    print(e)

#### Creating the inventory sets table according to the defined schema

In [16]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS inventory_sets\
        (inventory_id INTEGER,\
        set_number VARCHAR(10),\
        quantity INTEGER);")
except psycopg2.Error as e:
    print(e)

#### Creating the inventory parts table according to the defined schema

In [17]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS inventory_parts\
        (inventory_id INTEGER,\
        part_number VARCHAR(50),\
        colour_id INTEGER,\
        quantity INTEGER,\
        is_spare BOOLEAN);")
except psycopg2.Error as e:
    print(e)

#### Adding foreign key constraints to required tables

In [20]:
# Adding a foreign key constraint to the inventories table referencing the sets table
try:
    cur.execute("ALTER TABLE inventories \
                    ADD CONSTRAINT fk_set\
                    FOREIGN KEY(set_number)\
                    REFERENCES sets(set_number)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)
    
# Adding a foreign key constraint to the sets table referencing the themes table
try:
    cur.execute("ALTER TABLE sets \
                    ADD CONSTRAINT fk_theme\
                    FOREIGN KEY(theme_id)\
                    REFERENCES themes(id)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)

# Adding a foreign key constraint to the parts table referencing the parts categories table
try:
    cur.execute("ALTER TABLE parts \
                    ADD CONSTRAINT fk_partcat\
                    FOREIGN KEY(part_cat_id)\
                    REFERENCES part_categories(id)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)
    
# Adding a foreign key constraint to the inventory parts table referencing the inventories categories table
try:
    cur.execute("ALTER TABLE inventory_parts \
                    ADD CONSTRAINT fk_inventory\
                    FOREIGN KEY(inventory_id)\
                    REFERENCES inventories(id)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)
    
# Adding a foreign key constraint to the inventory parts table referencing the parts table
try:
    cur.execute("ALTER TABLE inventory_parts \
                    ADD CONSTRAINT fk_partnum\
                    FOREIGN KEY(part_number)\
                    REFERENCES parts(part_number)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)
    
# Adding a foreign key constraint to the inventory parts table referencing the colours table
try:
    cur.execute("ALTER TABLE inventory_parts \
                    ADD CONSTRAINT fk_colourid\
                    FOREIGN KEY(colour_id)\
                    REFERENCES colours(id)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)
    
# Adding a foreign key constraint to the inventory sets table referencing the inventories table
try:
    cur.execute("ALTER TABLE inventory_sets \
                    ADD CONSTRAINT fk_inventoryid\
                    FOREIGN KEY(inventory_id)\
                    REFERENCES inventories(id)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)
    
# Adding a foreign key constraint to the inventory sets table referencing the sets table
try:
    cur.execute("ALTER TABLE inventory_sets \
                    ADD CONSTRAINT fk_setnum\
                    FOREIGN KEY(set_number)\
                    REFERENCES sets(set_number)\
                    ON DELETE CASCADE;")
except psycopg2.Error as e:
    print(e)

#### Reading the csv files into pandas dataframes and importing the dataframes into their corresponding tables

In [21]:
# Defining the 'execute_df_values' function

def execute_df_values(conn, df, table): 

	# The function converts the DataFrame `df` into a list of tuples (`tuples`), where each tuple represents a row in the DataFrame
	tuples = [tuple(x) for x in df.to_numpy()] 

	# Constructing a comma-separated string of column names from the DataFrame.
	cols = ','.join(list(df.columns)) 

 
	# SQL query to insert data into the specific
	query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
 
	try: 
		extras.execute_values(cur, query, tuples) 
	except (Exception, psycopg2.DatabaseError) as e: 
		print(e) 
		conn.rollback() 
		return 

	print("the dataframe is inserted") 


In [22]:
# Defining the 'get_column_names' function  
 
def get_column_names(table_name):
    
    # The function returns the column names for specific tables in the database
    try:
        # Query to get column names from information_schema.columns
        query = f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}';"
        
        # Execute the query
        cur.execute(query)
        
        # Fetch all the results as a list of tuples and extract column names
        db_column_names = [column[0] for column in cur.fetchall()]
        
        return db_column_names
    except psycopg2.Error as e:
        print(e)


#### Inserting data into the part categories table

In [33]:
# Reading the part categories data in a dataframe
part_categories_df = pd.read_csv('part_categories.csv')

# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, part_categories_df, 'part_categories')

the dataframe is inserted


#### Inserting data into the parts table
The 'part_num' column in the dataframe has to have the same column name as the 'part_number' column in the database table.

In [34]:
# Reading the part categories data in a dataframe
parts_df = pd.read_csv('parts.csv')

# Renaming the part_num column in the dataframe to correspond with the part_number column in the parts table
column_mapping = {
    'part_num': 'part_number'
}

parts_df.rename(columns=column_mapping, inplace=True)

# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, parts_df, 'parts')

the dataframe is inserted


#### Inserting data into the colours table
The 'f' and 't' values in the 'is_trans' column need to be converted to their boolean alternatives before being inserted into the colours table. 

In [42]:
# Reading the colours data into a dataframe
colours_df = pd.read_csv('colors.csv')

# Replacing the respective values with Boolean False/True
colours_df.loc[colours_df['is_trans'] == 'f', 'is_trans'] = False
colours_df.loc[colours_df['is_trans'] == 't', 'is_trans'] = True

# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, colours_df, 'colours')

the dataframe is inserted


#### Inserting data into the themes table

The 'parent_id' column contains some missing values. These values have to be taken care of so that the data could be successfully inserted into the themes table. 

In [56]:
# Reading the part categories data in a dataframe
themes_df = pd.read_csv('themes.csv')

# Converting NaN values to Zero, and casting the dtype of the column to an integer
themes_df['parent_id'] = themes_df['parent_id'].fillna(0).astype(int)

# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, themes_df, 'themes')

the dataframe is inserted


In [57]:
# Reading the sets data in a dataframe
sets_df = pd.read_csv('sets.csv')

# Renaming the set_num column in the dataframe to correspond with the set_number column in the sets table
column_mapping = {
    'set_num': 'set_number'
}

sets_df.rename(columns=column_mapping, inplace=True)

# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, sets_df, 'sets')

Error: value too long for type character varying(50)



The error above indicates that some data to be inserted exceeds the data type constraint which represents VARCHAR(50) and which was set on the 'name' column. The data type for the said column needs to be changed to accomodate the data to be inserted.

In [58]:
# Updating the data type of the 'name' column in the sets table
try:
    cur.execute("ALTER TABLE sets\
                ALTER COLUMN name\
                TYPE VARCHAR;")
except psycopg2.Error as e:
    print(e)

In [59]:
# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, sets_df, 'sets')

Error: value too long for type character varying(10)



The error above indicates that some data to be inserted exceeds the data type constraint which represents VARCHAR(10) and which was set on the 'set_number' column. The data type for the said column needs to be changed to accomodate the data to be inserted.

In [60]:
# Updating the data type of the 'set_number' column in the sets table
try:
    cur.execute("ALTER TABLE sets\
                ALTER COLUMN set_number\
                TYPE VARCHAR;")
except psycopg2.Error as e:
    print(e)

In [64]:
# Inserting the data from the dataframe into the corresponding table
execute_df_values(conn, sets_df, 'sets')

the dataframe is inserted
